In [1]:
import numpy as np
import GCRCatalogs
from astropy.table import Table

from clmm.structures import GalaxyCluster

In [2]:
# fyi this is cluster.py

import pickle
from astropy.table import Table

class GalaxyCluster():
    def __init__(self, id=None, 
                       ra=None, dec=None,
                       z=None, richness=None,
                       gals=Table()
                ):
        self.id = id
        self.ra = ra
        self.dec = dec
        self.z = z
        self.richness = richness
        self.gals = gals

    def save(self, filename, **kwargs):
        with open(filename, 'wb') as f:
            pickle.dump(self, f, protocol=3, **kwargs)

    def load(self, filename, **kwargs):
        with open(filename, 'rb') as f:
            new_cl = pickle.load(f, **kwargs)
        self.__dict__.update(new_cl.__dict__)

In [3]:
# fyi this is load_from_DC2.py

import os
import numpy as np
import GCRCatalogs
from astropy.table import Table

from clmm.structures import GalaxyCluster

def load_from_dc2(N, catalog, save_dir):
    catalog = GCRCatalogs.load_catalog(catalog)
    halos = catalog.get_quantities(['galaxy_id', 'halo_mass', 'redshift','ra', 'dec'],
                                   filters=['halo_mass > 1e14','is_central==True'])

    for _ in range(N):
        i = np.random.randint(len(halos))

        id_cl = halos['galaxy_id'][i]
        ra_cl = halos['ra'][i]
        dec_cl = halos['dec'][i]
        z_cl = halos['redshift'][i]
        m_cl = halos['halo_mass'][i]

        # get galaxies around cluster
        ra_min, ra_max = ra_cl-0.3, ra_cl+0.3
        dec_min, dec_max = dec_cl-0.3, dec_cl+0.3
        z_min = z_cl + 0.1
        z_max = 1.5

        coord_filters = [
            'ra >= %s'%ra_min,
            'ra < %s'%ra_max,
            'dec >= %s'%dec_min,
            'dec < %s'%dec_max,
        ]
        z_filters = ['redshift >= %s'%z_min,
                     'redshift < %s'%z_max]

        gals = catalog.get_quantities(['galaxy_id', 'ra', 'dec', 'shear_1', 'shear_2',
                                       'redshift', 'convergence'], filters=(coord_filters + z_filters))

        # store the results into an astropy table
        t = Table([gals['galaxy_id'],gals['ra'],gals['dec'],gals['shear_1'],
                   gals['shear_2'],gals['redshift'],gals['convergence']],
                  names=('id','ra','dec', 'gamma1', 'gamma2', 'z', 'kappa'))


        c = GalaxyCluster(id=id_cl, ra=ra_cl, dec=dec_cl,
                    z=z_cl, richness=None,
                    gals=t)

        c.save(os.path.join(save_dir, '%s.p'%id_cl))



In [4]:
catalog = 'proto-dc2_v2.1.2_test'
save_dir = '/global/homes/m/mho1/scratch/clmm_scratch/cl_from_gcr'

load_from_dc2(5, catalog, save_dir)

/global/u1/m/mho1/desc/CLMM/examples/GCRCatalogs/alphaq.py:105: UserWarning: No md5 sum specified in the config file
  warnings.warn('No md5 sum specified in the config file')


In [5]:
load_file = '/global/homes/m/mho1/scratch/clmm_scratch/cl_from_gcr/443618.p'

cl = GalaxyCluster()
cl.load(load_file)

print('ra: %s, dec: %s'%(cl.ra, cl.dec))
cl.gals

ra: -0.320841, dec: -0.158532


id,ra,dec,gamma1,gamma2,z,kappa
int64,float32,float32,float32,float32,float64,float32
206753,-0.509809,-0.412629,0.00284167,0.0110131,0.478103539295,0.00148338
206983,-0.514711,-0.318065,1.05667e-05,-5.55245e-05,0.478475201685,0.00171316
217473,-0.586691,-0.346806,-0.00842941,-0.000413947,0.558600139463,-0.00416899
217477,-0.586256,-0.348024,-0.00824594,-0.000356569,0.558726541973,-0.00439441
217479,-0.42406,-0.392871,0.000648918,0.00673248,0.557891232526,0.00414896
217480,-0.507499,-0.408799,0.00359056,0.0117054,0.55874428609,0.0018031
217481,-0.449362,-0.399487,0.00429097,-0.00317511,0.558066987067,0.0106788
217482,-0.586198,-0.372236,-0.00425679,0.00217683,0.558867553352,-0.00675666
217484,-0.510224,-0.406202,0.00304988,0.011796,0.558772899401,0.00138766
